In [1]:
import numpy as np
import pandas as pd 

In [15]:
abalone=pd.read_csv("abalone.txt", sep=",", header=None, names=['sex','length','diameter','height','whole_weight','shucked_sweight','viscera_weight','shell_weight','rings'])
abalone

,sex,length,diameter,height,whole_weight,shucked_sweight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [16]:
abalone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
sex                4177 non-null object
length             4177 non-null float64
diameter           4177 non-null float64
height             4177 non-null float64
whole_weight       4177 non-null float64
shucked_sweight    4177 non-null float64
viscera_weight     4177 non-null float64
shell_weight       4177 non-null float64
rings              4177 non-null int64
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [21]:
abalone['Male']=np.where(abalone['sex']=='M', 1, 0)
abalone['Infant']=np.where(abalone['sex']=='I', 1, 0)
abalone['Female']=np.where(abalone['sex']=='F', 1, 0)

In [22]:
abalone.columns

Index(['sex', 'length', 'diameter', 'height', 'whole_weight',
       'shucked_sweight', 'viscera_weight', 'shell_weight', 'rings', 'Male',
       'Infant', 'Female'],
      dtype='object')

In [5]:
from sklearn.model_selection import train_test_split

In [23]:
xTrain, xTest, yTrain, yTest=train_test_split(abalone.drop(['sex','rings'], axis=1), abalone['rings'], test_size=0.3, random_state=20201005)
print(xTrain.shape)
print(yTrain.shape)

(2923, 10)
(2923,)


In [7]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
model=RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=20201005)

In [19]:
yTrain.isnull().sum()

0

In [25]:
model.fit(xTrain, yTrain)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=False,
                      random_state=20201005, verbose=0, warm_start=False)

In [27]:
yPred=np.round(model.predict(xTest))
yPred

array([10.,  9., 10., ...,  6., 10., 10.])

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
from sklearn.metrics import make_scorer

In [30]:
def rmsle(pv,av):
    pv=np.array(pv)
    av=np.array(av)
    log_predict=np.log(pv+1)
    log_actual=np.log(av+1)
    res=log_predict-log_actual
    res=np.square(res)
    mean_res=res.mean()
    score=np.sqrt(mean_res)
    return score
rmsle_scorer=make_scorer(rmsle)

In [32]:
score=cross_val_score(model, xTrain, yTrain, scoring=rmsle_scorer)
score.mean()

0.18003273108660237